In [1]:
# import every tool you might need for the project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import tweepy
import json
import os
from io import BytesIO
from PIL import Image

## Gather

In [2]:
df = pd.read_csv("twitter-archive.csv")
df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [3]:
# create a directory first then save images in the directory
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

with open(os.path.join(folder_name,
                      url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [4]:
#open and asign the downloaded tsv file to images
images = pd.read_csv('image_predictions/image-predictions.tsv', delimiter='\t')

In [5]:
# my personal API keys, secrets, and tokens which i m going to hide later before submission
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

In [6]:
# tweepy authentication process

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [7]:
# for step 3, get tweets info using list of tweet ids and record the failed id's then read into a .txt file

tweet_ids = list(df.tweet_id)
tweet_data = {}

for tweet in tweet_ids:
    try:
        tweet_status = api.get_status(tweet, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        tweet_data[str(tweet)] = tweet_status._json
    except: 
        print("Error: " + str(tweet))

Error: 888202515573088257
Error: 873697596434513921
Error: 872668790621863937
Error: 869988702071779329
Error: 866816280283807744
Error: 861769973181624320
Error: 845459076796616705
Error: 842892208864923648
Error: 837012587749474308
Error: 827228250799742977
Error: 812747805718642688
Error: 802247111496568832
Error: 775096608509886464
Error: 770743923962707968


Rate limit reached. Sleeping for: 642


Error: 754011816964026368
Error: 680055455951884288


Rate limit reached. Sleeping for: 664


In [8]:
with open('tweet_json.txt', 'w', encoding='utf-8') as outfile:  
    json.dump(tweet_data, outfile, sort_keys = True, indent=4, ensure_ascii = False)

In [9]:
# read the json file with index set as the orientation
df_tweet = pd.read_json('tweet_json.txt', orient='index')

### Assess

In [10]:
df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [11]:
df_tweet.head(2)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
1991-02-08 13:48:08.022790149,NaN,NaN,2015-11-15 22:32:08,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/BL...,2530,0,NaN,666020888022790144,666020888022790144,...,NaN,NaN,NaN,501,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,0,"{'contributors_enabled': False, 'created_at': ..."
1991-02-08 16:08:05.002620928,NaN,NaN,2015-11-15 23:05:30,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/r7...,126,0,NaN,666029285002620928,666029285002620928,...,NaN,NaN,NaN,47,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,0,"{'contributors_enabled': False, 'created_at': ..."


In [12]:
images.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [14]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2340 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2340 non-null datetime64[ns]
entities                         2340 non-null object
extended_entities                1822 non-null object
favorite_count                   2340 non-null int64
favorited                        2340 non-null int64
geo                              0 non-null float64
id                               2340 non-null int64
id_str                           2340 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  23

In [15]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [16]:
df[df.isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1621,684926975086034944,NaN,NaN,2016-01-07 02:38:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Bruiser &amp; Charlie. They are the best ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684926975...,11,10,Bruiser,None,None,None,None
888,759846353224826880,NaN,NaN,2016-07-31 20:21:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kirby. He's a Beneblip Cumberpat. Pret...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/759846353...,11,10,Kirby,None,None,None,None
1231,713177543487135744,NaN,NaN,2016-03-25 01:35:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lance. Lance doesn't give a shit. 10/1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713177543...,10,10,Lance,None,None,None,None
2196,668852170888998912,NaN,NaN,2015-11-23 18:02:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Bobb. Bobb is a Golden High Fescu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668852170...,11,10,Bobb,None,None,None,None
1076,739606147276148736,NaN,NaN,2016-06-05 23:53:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Benji. He just turned 1. Has already give...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/739606147...,9,10,Benji,None,None,pupper,None


In [18]:
df['name'].value_counts()

None           745
a               55
Charlie         12
Oliver          11
Cooper          11
Lucy            11
Tucker          10
Penny           10
Lola            10
Bo               9
Winston          9
Sadie            8
the              8
Buddy            7
Daisy            7
Bailey           7
Toby             7
an               7
Scout            6
Leo              6
Stanley          6
Dave             6
Jax              6
Milo             6
Bella            6
Koda             6
Jack             6
Rusty            6
Oscar            6
George           5
              ... 
Stephanus        1
Molly            1
Coleman          1
Josep            1
Halo             1
Ronnie           1
Sonny            1
Koko             1
Einstein         1
Dante            1
Cuddles          1
Fabio            1
Godi             1
Tom              1
Herb             1
Lassie           1
Randall          1
Carper           1
Clybe            1
Shakespeare      1
Rontu            1
Bobby       

In [19]:
df_tweet.sample(5)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
1996-08-18 12:18:01.858686976,NaN,NaN,2017-03-11 01:15:58,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/76...,17187,0,NaN,840370681858686976,840370681858686976,...,NaN,NaN,NaN,4854,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",You have been visited by the magical sugar jar...,0,"{'contributors_enabled': False, 'created_at': ..."
1993-02-24 17:03:03.004487680,NaN,NaN,2016-05-12 01:40:42,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/Cn...,5131,0,NaN,730573383004487680,730573383004487680,...,NaN,NaN,NaN,2290,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Rooney. He can't comprehend glass. 10/...,0,"{'contributors_enabled': False, 'created_at': ..."
1991-04-25 14:54:31.085670400,NaN,NaN,2015-12-04 01:40:29,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/Pa...,742,0,NaN,672591271085670400,672591271085670400,...,NaN,NaN,NaN,189,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Lots of pups here. All are Judea Hazelnuts. Ex...,0,"{'contributors_enabled': False, 'created_at': ..."
1996-02-07 13:16:42.998870016,NaN,NaN,2017-01-24 01:08:40,"{'hashtags': [], 'symbols': [], 'urls': [{'dis...",NaN,13326,0,NaN,823699002998870016,823699002998870016,...,NaN,NaN,NaN,2625,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Winston. The goggles make him a superh...,1,"{'contributors_enabled': False, 'created_at': ..."
1992-03-08 04:34:44.593901568,NaN,NaN,2016-02-17 18:49:22,"{'hashtags': [], 'media': [{'display_url': 'pi...",{'media': [{'display_url': 'pic.twitter.com/8N...,2163,0,NaN,700029284593901568,700029284593901568,...,NaN,NaN,NaN,622,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Coops. His ship is taking on water. So...,0,"{'contributors_enabled': False, 'created_at': ..."


In [20]:
df.shape[0] - df[df['retweeted_status_id'].isnull()].shape[0]

181

In [21]:
images.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
100,667782464991965184,https://pbs.twimg.com/media/CURwm3cUkAARcO6.jpg,1,lorikeet,0.466149,False,hummingbird,0.083011,False,African_grey,0.054247,False
959,705591895322394625,https://pbs.twimg.com/media/CcrEFQdUcAA7CJf.jpg,1,basenji,0.877207,True,Italian_greyhound,0.047854,True,miniature_pinscher,0.035638,True
943,704347321748819968,https://pbs.twimg.com/media/CcZYJniXEAAEJRF.jpg,1,teddy,0.233378,False,feather_boa,0.088474,False,Brittany_spaniel,0.082917,True
1262,748977405889503236,https://pbs.twimg.com/media/CmTm-XQXEAAEyN6.jpg,1,German_short-haired_pointer,0.742216,True,bluetick,0.152810,True,English_setter,0.051835,True
309,671538301157904385,https://pbs.twimg.com/media/CVHIhi2WsAEgdKk.jpg,1,park_bench,0.194211,False,water_bottle,0.071870,False,beacon,0.053433,False


In [22]:
df_tweet.shape[0] - df_tweet[df_tweet['retweeted_status'].isnull()].shape[0]

167

### Quality

`df table:`
- some of the dog names contain normal words like 'actually', 'this', 'infuriating', 'getting', and so on. I will replace them with none.
- remove the retweets that are approximately 181 rows
- remove any columns that are not useful to the project

`Image predictions table:`
- Consistency issues with 3 columns
- p1: capitalize first letter
- p2: capitalize first letter
- p3: capitalize first letter
- p1: replace '_' with space
- p2: replace '_' with space
- p3: replace '_' with space

`df_tweet table:`
- remove the 176 rows of retweeted status id
- rename columns that mismatch names of other tables for merging purposes later like 'id' column

### Tidiness

`df table:`
- Date time columns need to be converted to datetime objects
- Rating column needs to converted to floats
- drop any columns that are not useful to the project
- combine dog stages like "pupper", "doggo" and the like into one column

`tweet_df table:`
- convert id column to string since its ids
- use the tweet_id as the index

- merge this table using join with the rest

### Clean `df_clean` table first

In [23]:
# create a separate copy of all 3 dataframes
df_clean = df.copy()
df_tweet_clean = df_tweet.copy()
images_clean = images.copy()

#### Define
- some of the dog names contain normal words like 'actually', 'this', 'infuriating', 'getting', and so on. I will replace them with none.

#### Code

In [24]:
# start off by replacing invalid names
# used excel to scan them visually and will replace all the faulty ones
df_clean['name'].replace(['actually', 'getting', 'infuriating', 'the', 'light', 'an', 'life',
                         'this', 'very', 'not', 'one', 'O', 'quite', 'such', 'old', 'all', 'by', 'a'], 
                         ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
                         'None', 'None', 'None', 'None', 'None', 'None', 'None'], inplace=True)

#### Test

In [25]:
df_clean['name'].sample(50) # ran the sample code multiple times and did not find any wrong name

95         Walter
820            Al
19         Canela
1653         None
407       Winston
2263         Kial
761       Stanley
267         Penny
20         Gerald
1968      Frankie
1727     Penelope
1082         None
101          None
2182     Harrison
1501         None
872          None
329         Poppy
2118        Damon
1375        Bilbo
601          Lola
414        Hobbes
490          Gary
1294       Farfle
1855        Ellie
194        George
142      Meatball
131          None
1917        Kenny
192          None
435       Peaches
1173        Derek
2145         None
1013      Atticus
1932         None
827        DonDon
2047    Scruffers
1595         None
742          None
2067         None
2045          Jax
2282    Genevieve
648         Robin
2295         Gabe
725         Penny
1865         None
295       Vincent
2117         Herb
143       Paisley
1250     Chuckles
332          None
Name: name, dtype: object

#### Define
- remove the retweets that are approximately 181 rows

#### Code

In [26]:
# remove rewtweeted id's that are not null
ret_index = df_clean[df_clean['retweeted_status_id'].isnull()==False].index
df_clean.drop(ret_index, inplace=True)

#### Test

In [27]:
df_clean['retweeted_status_id'].isna().sum()

2175

#### Define
- Combine dog stages like "pupper", "doggo" and the like into one column

#### Code

In [28]:
# combine four into one
df_clean['stage'] = df[['doggo', 'floofer','pupper','puppo']].apply(lambda x: ''.join(x), axis=1)
#replace combined values with valied ones
df_clean['stage'].replace(['NoneNoneNoneNone', 'doggoNoneNoneNone', 'NoneflooferNoneNone', 'NoneNonepupperNone', 'NoneNoneNonepuppo'],
                          ['None', 'doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

In [29]:
# drop used columns
df_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], inplace=True, axis=1)

#### Test

In [30]:
df_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None


#### Define
- remove any other columns that are not useful to the project

#### Code

In [31]:
df_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'stage'],
      dtype='object')

In [32]:
df_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'text', 'retweeted_status_id',
              'retweeted_status_user_id','retweeted_status_timestamp', 'expanded_urls'], axis=1, inplace=True)

#### Test

In [33]:
df_clean.head()

,tweet_id,timestamp,rating_numerator,rating_denominator,name,stage
0,892420643555336193,2017-08-01 16:23:56 +0000,13,10,Phineas,None
1,892177421306343426,2017-08-01 00:17:27 +0000,13,10,Tilly,None
2,891815181378084864,2017-07-31 00:18:03 +0000,12,10,Archie,None
3,891689557279858688,2017-07-30 15:58:51 +0000,13,10,Darla,None
4,891327558926688256,2017-07-29 16:00:24 +0000,12,10,Franklin,None


#### Define
- convert timestamp column to datetime and extract date and time as separate columns

In [34]:
# convert to datetime
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

In [35]:
# extract date and time
# got stuck with this for a while so googled how to
df_clean['date'] = df_clean['timestamp'].apply(lambda time: time.strftime('%m-%d-%Y'))
df_clean['time'] = df_clean['timestamp'].apply(lambda time: time.strftime('%H:%M:%S'))

In [36]:
# convert date and time column to datetime
df_clean['date'] = pd.to_datetime(df_clean['date'])
df_clean['time'] = pd.to_datetime(df_clean['time'])

In [37]:
# drop used column
df_clean.drop('timestamp', axis=1, inplace=True)

#### Test

In [38]:
df_clean.head()

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time
0,892420643555336193,13,10,Phineas,None,2017-08-01,2019-01-21 16:23:56
1,892177421306343426,13,10,Tilly,None,2017-08-01,2019-01-21 00:17:27
2,891815181378084864,12,10,Archie,None,2017-07-31,2019-01-21 00:18:03
3,891689557279858688,13,10,Darla,None,2017-07-30,2019-01-21 15:58:51
4,891327558926688256,12,10,Franklin,None,2017-07-29,2019-01-21 16:00:24


#### Define
- tweet_id should be string even though the values are numbers

#### Code

In [39]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

#### Test

In [40]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 7 columns):
tweet_id              2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
stage                 2175 non-null object
date                  2175 non-null datetime64[ns]
time                  2175 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 135.9+ KB


## Clean `image_clean`

In [41]:
images_clean.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


#### Define
- remove underscores then capitalize first letters of p1, p2, and p3

In [42]:
# remove underscores
images_clean['p1'] = images_clean['p1'].str.replace('_', ' ')
images_clean['p2'] = images_clean['p2'].str.replace('_', ' ')
images_clean['p3'] = images_clean['p3'].str.replace('_', ' ')

# capitalize first letters
images_clean['p1'] = images_clean['p1'].str.title()
images_clean['p2'] = images_clean['p2'].str.title()
images_clean['p3'] = images_clean['p3'].str.title()

#### Test

In [43]:
images_clean.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True


#### Define
- convert tweet_id to string

#### Code

In [44]:
images_clean['tweet_id'] = images_clean['tweet_id'].astype(str)

#### Test

In [45]:
images_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


### Clean `df_tweet_clean`

#### Define
- rename 'id' column to 'tweet_id' to make joining possible later

#### Code

In [46]:
df_tweet_clean.rename(columns={'id' : 'tweet_id'}, inplace=True)

#### Test

In [47]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2340 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2340 non-null datetime64[ns]
entities                         2340 non-null object
extended_entities                1822 non-null object
favorite_count                   2340 non-null int64
favorited                        2340 non-null int64
geo                              0 non-null float64
tweet_id                         2340 non-null int64
id_str                           2340 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  23

#### Define
- remove retweeted_status to get rid of non original tweets

In [49]:
df_tweet_ret_index = df_tweet_clean[df_tweet_clean['retweeted_status'].isnull()==False].index
df_tweet_clean.drop(df_tweet_ret_index, inplace=True)

#### Test

In [50]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2173 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2173 non-null datetime64[ns]
entities                         2173 non-null object
extended_entities                1747 non-null object
favorite_count                   2173 non-null int64
favorited                        2173 non-null int64
geo                              0 non-null float64
tweet_id                         2173 non-null int64
id_str                           2173 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  21

#### Define
- Remove columns not needed for the project

#### Code

In [51]:
df_tweet_clean.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'extended_entities', 'favorite_count', 'favorited', 'geo', 'tweet_id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'possibly_sensitive_appealable', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'retweet_count',
       'retweeted', 'retweeted_status', 'source', 'text', 'truncated', 'user'],
      dtype='object')

In [52]:
df_tweet_clean.drop(['contributors', 'coordinates', 'created_at', 'entities', 'extended_entities',
                    'favorited', 'geo', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 
                    'in_reply_to_user_id_str','is_quote_status', 'lang', 'place', 
                    'possibly_sensitive', 'possibly_sensitive_appealable',
                    'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'retweeted', 'retweeted_status',
                    'source', 'truncated', 'user'], axis=1, inplace=True)

#### Test

In [53]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2173 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 4 columns):
favorite_count    2173 non-null int64
tweet_id          2173 non-null int64
retweet_count     2173 non-null int64
text              2173 non-null object
dtypes: int64(3), object(1)
memory usage: 84.9+ KB


#### Define
- tweet_id column needs to be string

#### Code

In [54]:
df_tweet_clean['tweet_id'] = df_tweet_clean['tweet_id'].astype(str)

#### Test

In [55]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2173 entries, 1991-02-08 13:48:08.022790149 to 1998-04-12 22:37:23.555336193
Data columns (total 4 columns):
favorite_count    2173 non-null int64
tweet_id          2173 non-null object
retweet_count     2173 non-null int64
text              2173 non-null object
dtypes: int64(2), object(2)
memory usage: 84.9+ KB


### Clean All Tables

#### Define
- merge using inner join
- fix up by dropping any unneeded columns as well as datatypes

#### Code

In [56]:
df_merged = pd.merge(df_clean, df_tweet_clean, on='tweet_id')
df_merged = pd.merge(df_merged, images_clean, on='tweet_id')

#### Test

In [57]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298 entries, 0 to 1297
Data columns (total 21 columns):
tweet_id              1298 non-null object
rating_numerator      1298 non-null int64
rating_denominator    1298 non-null int64
name                  1298 non-null object
stage                 1298 non-null object
date                  1298 non-null datetime64[ns]
time                  1298 non-null datetime64[ns]
favorite_count        1298 non-null int64
retweet_count         1298 non-null int64
text                  1298 non-null object
jpg_url               1298 non-null object
img_num               1298 non-null int64
p1                    1298 non-null object
p1_conf               1298 non-null float64
p1_dog                1298 non-null bool
p2                    1298 non-null object
p2_conf               1298 non-null float64
p2_dog                1298 non-null bool
p3                    1298 non-null object
p3_conf               1298 non-null float64
p3_dog                1

In [58]:
df_merged.head(3)

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,891815181378084864,12,10,Archie,None,2017-07-31,2019-01-21 00:18:03,24552,4064,This is Archie. He is a rare Norwegian Pouncin...,...,1,Chihuahua,0.716012,True,Malamute,0.078253,True,Kelpie,0.031379,True
1,891689557279858688,13,10,Darla,None,2017-07-30,2019-01-21 15:58:51,41346,8447,This is Darla. She commenced a snooze mid meal...,...,1,Paper Towel,0.170278,False,Labrador Retriever,0.168086,True,Spatula,0.040836,False
2,891327558926688256,12,10,Franklin,None,2017-07-29,2019-01-21 16:00:24,39531,9154,This is Franklin. He would like you to stop ca...,...,2,Basset,0.555712,True,English Springer,0.225770,True,German Short-Haired Pointer,0.175219,True


#### Define
- rating numerator to float
- rating denominator to float
- remove year from time

In [59]:
df_merged['rating_numerator'] = df_merged['rating_numerator'].astype(float)
df_merged['rating_denominator'] = df_merged['rating_denominator'].astype(float)
df_merged['time'] = df_merged['time'].apply(lambda time: time.strftime('%H:%M:%S'))

#### Test

In [60]:
df_merged.head(2)

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,891815181378084864,12.0,10.0,Archie,None,2017-07-31,00:18:03,24552,4064,This is Archie. He is a rare Norwegian Pouncin...,...,1,Chihuahua,0.716012,True,Malamute,0.078253,True,Kelpie,0.031379,True
1,891689557279858688,13.0,10.0,Darla,None,2017-07-30,15:58:51,41346,8447,This is Darla. She commenced a snooze mid meal...,...,1,Paper Towel,0.170278,False,Labrador Retriever,0.168086,True,Spatula,0.040836,False


In [61]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298 entries, 0 to 1297
Data columns (total 21 columns):
tweet_id              1298 non-null object
rating_numerator      1298 non-null float64
rating_denominator    1298 non-null float64
name                  1298 non-null object
stage                 1298 non-null object
date                  1298 non-null datetime64[ns]
time                  1298 non-null object
favorite_count        1298 non-null int64
retweet_count         1298 non-null int64
text                  1298 non-null object
jpg_url               1298 non-null object
img_num               1298 non-null int64
p1                    1298 non-null object
p1_conf               1298 non-null float64
p1_dog                1298 non-null bool
p2                    1298 non-null object
p2_conf               1298 non-null float64
p2_dog                1298 non-null bool
p3                    1298 non-null object
p3_conf               1298 non-null float64
p3_dog                1298 

### Analysis

In [68]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298 entries, 0 to 1297
Data columns (total 21 columns):
tweet_id              1298 non-null object
rating_numerator      1298 non-null float64
rating_denominator    1298 non-null float64
name                  1298 non-null object
stage                 1298 non-null object
date                  1298 non-null datetime64[ns]
time                  1298 non-null object
favorite_count        1298 non-null int64
retweet_count         1298 non-null int64
text                  1298 non-null object
jpg_url               1298 non-null object
img_num               1298 non-null int64
p1                    1298 non-null object
p1_conf               1298 non-null float64
p1_dog                1298 non-null bool
p2                    1298 non-null object
p2_conf               1298 non-null float64
p2_dog                1298 non-null bool
p3                    1298 non-null object
p3_conf               1298 non-null float64
p3_dog                1298 

In [69]:
# use describe to get a picture of the numbers
df_merged.describe()

,rating_numerator,rating_denominator,favorite_count,retweet_count,img_num,p1_conf,p2_conf,p3_conf
count,1298.000000,1298.000000,1298.000000,1298.000000,1298.000000,1298.000000,1.298000e+03,1.298000e+03
mean,12.845917,10.546225,8144.900616,2459.197227,1.187211,0.586720,1.371536e-01,6.149405e-02
std,51.167293,7.877519,11400.865346,3978.552367,0.540929,0.273508,1.018387e-01,5.202003e-02
min,1.000000,2.000000,78.000000,12.000000,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,10.000000,10.000000,1681.000000,563.500000,1.000000,0.354689,5.443702e-02,1.651733e-02
50%,11.000000,10.000000,3751.500000,1226.500000,1.000000,0.576936,1.203825e-01,4.961540e-02
75%,12.000000,10.000000,10048.250000,2908.000000,1.000000,0.836617,1.983982e-01,9.471785e-02
max,1776.000000,170.000000,126914.000000,60923.000000,4.000000,1.000000,4.676780e-01,2.710420e-01


#### takeaways to just be observed
- rating_numerator has a max value of 1776
- rating_denominator has a min value of 2 and max value of 170
- img_num has a max of 4 when up to 75th percentile its only 1
- p1_conf has a perfect accuracy as max

In [70]:
df_merged[df_merged['rating_denominator'] == 170]

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
532,731156023742988288,204.0,170.0,None,None,2016-05-13,16:15:54,4004,1351,Say hello to this unbelievably well behaved sq...,...,1,Lakeside,0.501767,False,Breakwater,0.051351,False,King Penguin,0.049444,False


In [71]:
df_merged[df_merged['rating_numerator'] == 1776]
# atticus received highest rating

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
451,749981277374128128,1776.0,10.0,Atticus,None,2016-07-04,15:00:45,5377,2632,This is Atticus. He's quite simply America af....,...,1,Bow Tie,0.533941,False,Sunglasses,0.080822,False,Sunglass,0.050776,False


In [72]:
df_merged[df_merged['p1_conf'] == 1]
# 100% confidence in jigsaw puzzle seems off

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1224,667866724293877760,10.0,10.0,Shaggy,None,2015-11-21,00:46:50,3028,1052,This is Shaggy. He knows exactly how to solve ...,...,1,Jigsaw Puzzle,1.0,False,Prayer Rug,1.011300e-08,False,Doormat,1.740170e-10,False


In [73]:
df_merged[df_merged['img_num'] > 3]
# seems ok

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
22,885167619883638784,13.0,10.0,None,None,2017-07-12,16:03:00,21507,4291,Here we have a corgi undercover as a malamute....,...,4,Malamute,0.812482,True,Siberian Husky,0.071712,True,Eskimo Dog,0.055770,True
74,864197398364647424,13.0,10.0,Paisley,None,2017-05-15,19:14:50,30143,9005,This is Paisley. She ate a flower just to prov...,...,4,Golden Retriever,0.945905,True,Labrador Retriever,0.021264,True,Tibetan Mastiff,0.020493,True
95,856282028240666624,14.0,10.0,Cermet,None,2017-04-23,23:01:59,28106,6499,"This is Cermet, Paesh, and Morple. They are ab...",...,4,Chihuahua,0.876543,True,Italian Greyhound,0.032962,True,Cardigan,0.020776,True
110,849412302885593088,12.0,10.0,Noosh,None,2017-04-05,00:04:08,16379,3302,This is Noosh. He noticed you were in the show...,...,4,Schipperke,0.907559,True,Crossword Puzzle,0.017934,False,Chihuahua,0.016191,True
138,836001077879255040,13.0,10.0,None,None,2017-02-26,23:52:43,20023,4602,Atlas is back and this time he's prettier than...,...,4,Samoyed,0.963558,True,White Wolf,0.019848,False,Malamute,0.005904,True
153,831315979191906304,13.0,10.0,None,pupper,2017-02-14,01:35:49,6872,1204,I couldn't make it to the #WKCDogShow BUT I ha...,...,4,Briard,0.982755,True,Soft-Coated Wheaten Terrier,0.009084,True,Bouvier Des Flandres,0.004693,True
155,830097400375152640,13.0,10.0,Samson,None,2017-02-10,16:53:37,10352,3244,Meet Samson. He's absolute fluffy perfection. ...,...,4,Toy Poodle,0.442713,True,Pomeranian,0.142073,True,Pekinese,0.125745,True
166,826848821049180160,13.0,10.0,Cupid,doggo,2017-02-01,17:44:55,38522,11062,This is Cupid. He was found in the trash. Now ...,...,4,Great Pyrenees,0.858764,True,Golden Retriever,0.023526,True,Pekinese,0.017104,True
281,791406955684368384,12.0,10.0,Levi,None,2016-10-26,22:31:36,14042,4513,Say hello to Levi. He's a Madagascan Butterbop...,...,4,Pembroke,0.972629,True,Cardigan,0.027026,True,Basenji,0.000153,True
307,783391753726550016,13.0,10.0,Godi,None,2016-10-04,19:42:03,17996,6012,Meet Godi. He's an avid beachgoer and part tim...,...,4,Norwegian Elkhound,0.877130,True,Cairn,0.086241,True,Keeshond,0.011019,True


#### Questions to analyze
- What is most common dog name?
- Most retweeted dogs?
- Most favorited dogs?

In [74]:
# what is most common dog name? Oliver.
df_merged['name'].value_counts()

None         430
Oliver         8
Winston        7
Tucker         7
Lucy           6
Penny          6
Cooper         6
Sadie          5
Toby           5
Bailey         5
Lola           5
Bella          5
Clark          4
Clarence       4
Sophie         4
Dexter         4
Shadow         3
Daisy          3
Riley          3
Rusty          3
Bo             3
Stanley        3
Maggie         3
Milo           3
Malcolm        3
Winnie         3
Jimothy        3
Finn           3
Cassie         3
Koda           3
            ... 
Hubertson      1
Cheesy         1
William        1
Amy            1
Bloo           1
Vinnie         1
Moose          1
Margo          1
Bookstore      1
Vixen          1
Levi           1
Kota           1
Filup          1
Covach         1
Chase          1
Kirk           1
Dylan          1
Kendall        1
Chloe          1
Sephie         1
Bell           1
Major          1
Kenny          1
Gilbert        1
Roosevelt      1
Julius         1
Derby          1
Tiger         

In [75]:
# Most favorited dogs. Stephan.
df_merged.groupby('name')['favorite_count'].max().sort_values(ascending=False)

name
Stephan       126914
Bo             92076
None           90512
Zoey           81851
Aja            78260
Kenneth        77282
Lilly          70461
Dexter         55881
Gary           49640
Bella          45059
Alfy           44096
Dewey          42020
Darla          41346
Emmy           41278
Luna           40261
Franklin       39531
Cupid          38522
Phil           37828
Hobbes         36207
Maisey         35311
Thor           34623
Maya           34475
Sebastian      34016
Leo            33462
Hurley         33123
Vixen          32866
Riley          32073
Jimbo          32027
Dave           31184
Paisley        30143
               ...  
Gòrdón           469
Biden            468
Bobb             459
Gin              447
Jaspers          446
Banditt          441
Nigel            439
Jo               420
Kloey            395
Maks             394
Olive            379
Anthony          379
Saydee           374
Ronduh           373
Dook             368
Josep            351
Raphael 

In [76]:
# Most retweeted dogs. Also Stephan.
df_merged.groupby('name')['retweet_count'].max().sort_values(ascending=False)

name
Stephan       60923
Bo            40036
None          33751
Kenneth       32612
Zoey          26066
Hurley        20078
Aja           18728
Lilly         18152
Dexter        17537
Phil          15400
Gary          15316
Lucy          13567
Dave          12791
Riley         11804
Leo           11457
Alfy          11353
Butter        11280
Cupid         11062
Luna          10965
Thor          10874
Hobbes        10845
Sebastian     10253
Moreton       10111
Dewey          9989
Tyrone         9742
Bella          9721
Vixen          9594
Franklin       9154
Paisley        9005
Rory           8735
              ...  
Andru           170
Snickers        162
Nigel           161
Skye            154
Olive           152
Jonah           146
Banditt         141
Maks            138
Jo              135
Gabe            128
Jazz            128
Saydee          127
Timofy          118
Kloey           114
Gin             114
Lambeau         103
Rufio            96
Raphael          95
Carll          

In [77]:
# Stephan is a Chihuahua
df_merged[df_merged['name'] == 'Stephan']

,tweet_id,rating_numerator,rating_denominator,name,stage,date,time,favorite_count,retweet_count,text,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
232,807106840509214720,13.0,10.0,Stephan,None,2016-12-09,06:17:20,126914,60923,This is Stephan. He just wants to help. 13/10 ...,...,1,Chihuahua,0.50537,True,Pomeranian,0.120358,True,Toy Terrier,0.077008,True


In [78]:
# What Stephan looks like. (link broken)
df_merged[df_merged['name'] == 'Stephan']['jpg_url']

232    https://pbs.twimg.com/ext_tw_video_thumb/80710...
Name: jpg_url, dtype: object

### Visualization
- see how strong the correlation is between favorites and retweets.